In [6]:
import pandas as pd

In [7]:
"""
Private
"""
class KI_PRIVATE:
    @staticmethod
    def get_app_key():
        app_key = "your_app_key"
        return app_key

    @staticmethod
    def get_app_secret():
        app_secret = "your_app_secret"
        return app_secret

    @staticmethod
    def get_acc_number():
        acc_number = "your_acc_number"
        return acc_number

from private import KI_PRIVATE

In [8]:
class DART_BROKER:
    def __init__(self, dart_private) -> None:
        self.dart_private = dart_private

    def __call__(self):
        import dart_fss as dart_broker

        dart_broker.set_api_key(
            api_key=self.dart_private.get_api_key(),
        )
        return dart_broker


class KI_BROKER:
    def __init__(self, ki_private) -> None:
        self.ki_private = ki_private

    def __call__(self):
        from mojito import KoreaInvestment

        ki_broker = KoreaInvestment(
            api_key=self.ki_private.get_app_key(),
            api_secret=self.ki_private.get_app_secret(),
            acc_no=self.ki_private.get_acc_number(),
        )
        return ki_broker

In [9]:
# ANALYSIS

buying_df = pd.read_csv("./data/dynamic/buying_df.csv", index_col=0)
buying_df["StockCode"] = buying_df["StockCode"].apply(lambda x: str(x).zfill(6))

In [10]:
ki_private = KI_PRIVATE()
ki_broker = KI_BROKER(ki_private)()

In [65]:
class ORDER_CONTROLLER:
    def __init__(self, ki_broker) -> None:
        self.ki_broker = ki_broker

    def get_current_cash(self):
        balance_resp = ki_broker.fetch_balance()
        current_cash = balance_resp["output2"][0]["dnca_tot_amt"]
        return current_cash

    def get_current_price(self, stockcode):
        resp = ki_broker.fetch_price(stockcode)["output"]
        current_price = resp["stck_prpr"]
        return current_price

    def get_price_dict(self, stockcodes):
        price_dict = dict()
        for stockcode in stockcodes:
            price_dict[stockcode] = self.get_current_price(stockcode)
        return price_dict

    def get_formed_df(self, buying_df):
        current_cash = self.get_current_cash()
        stock_current_cash = int(current_cash) / len(buying_df)

        stockcodes = sorted(set(buying_df["StockCode"]))
        price_dict = self.get_price_dict(stockcodes)

        buying_df["CurrentPrice"] = buying_df["StockCode"].map(price_dict)

        buying_df["Quantity"] = stock_current_cash // buying_df["CurrentPrice"].astype(int)
        buying_df = buying_df[buying_df["Quantity"] > 0]
        buying_df.loc[:, ["Quantity"]] = buying_df["Quantity"].apply(lambda x: str(int(x)))
        formed_df = buying_df.loc[:, ["StockCode", "Quantity"]]
        return formed_df

    def request_order(self, buying_df):
        formed_df = self.get_formed_df(buying_df)

        resp_dict = dict()
        for idx, row in formed_df.iterrows():
            try:
                ticker = row["StockCode"]
                quantity = row["Quantity"]
                resp = ki_broker.create_market_buy_order(symbol=ticker, quantity=quantity)
                resp_dict[ticker] = resp
            except:
                resp_dict[ticker] = "Failed"
        return resp_dict

In [63]:
order_controller = ORDER_CONTROLLER(ki_broker)
order_resq = order_controller.request_order(buying_df)
order_resq

{'003300': {'rt_cd': '0',
  'msg_cd': 'APBK0013',
  'msg1': '주문 전송 완료 되었습니다.',
  'output': {'KRX_FWDG_ORD_ORGNO': '91255',
   'ODNO': '0000030148',
   'ORD_TMD': '091517'}},
 '075970': {'rt_cd': '0',
  'msg_cd': 'APBK0013',
  'msg1': '주문 전송 완료 되었습니다.',
  'output': {'KRX_FWDG_ORD_ORGNO': '91255',
   'ODNO': '0000030149',
   'ORD_TMD': '091517'}}}